In [ ]:
from pdb import run
import matplotlib.pyplot as plt
import numpy as np
import json
from scipy.signal import savgol_filter
from scipy.ndimage import uniform_filter1d
import math
from scipy import stats

distance = ["12.5", "17.5", "21.5", "25.5", "29.5", "33.5", "37.5", "41.5", "45.5", "49.5"]

with open("c_ring_4_1.json", "r") as f:
    data = json.load(f)
    c_ring_4_1 = np.array([[complex(item["real"], item["imag"]) for item in row] for row in data])

with open("c_ring_6_05.json", "r") as f:
    data = json.load(f)
    c_ring_6_05 = np.array([[complex(item["real"], item["imag"]) for item in row] for row in data])

with open("c_ring_8_55.json", "r") as f:
    data = json.load(f)

c_ring_8_55 = np.array([[complex(item["real"], item["imag"]) for item in row] for row in data])

with open("c_ring_13_75.json", "r") as f:
    data = json.load(f)
    c_ring_13_75 = np.array([[complex(item["real"], item["imag"]) for item in row] for row in data])

with open("ssb.json", "r") as f:
    data = json.load(f)
    ssb = np.array([[complex(item["real"], item["imag"]) for item in row] for row in data])

with open("ssm.json", "r") as f:
    data = json.load(f)
    ssm = np.array([[complex(item["real"], item["imag"]) for item in row] for row in data])

with open("sss.json", "r") as f:
    data = json.load(f)
    sss = np.array([[complex(item["real"], item["imag"]) for item in row] for row in data])

measurements = [sss, ssm, ssb, c_ring_4_1, c_ring_6_05, c_ring_8_55, c_ring_13_75]
label = ["sss", "ssm", "ssb", "radius=4.1 cm", "radius=6.05 cm", "radius=8.55 cm", "radius=13.75 cm"]

def running_average_filter(data, N, smoothing):
    # Define the filter kernel
    smoothed_data = data

    if (smoothing):
        # smoothed_data = uniform_filter1d(data, size=N) # running mean
        smoothed_data = savgol_filter(data, window_length=N, polyorder=3) # savgol filter

    return smoothed_data

lim = 45
for i in range(len(measurements)):
    for k in range(len(measurements[i])):
        measurements[i][k].real[1:lim] = running_average_filter(measurements[i][k].real[1:lim], N=11, smoothing=True)
        measurements[i][k].imag[1:lim] = running_average_filter(measurements[i][k].imag[1:lim], N=11, smoothing=True)

def compareSizes(measurements, start, end):
    lim = 45
    fig, axs = plt.subplots(1, 2, figsize=(15, 9))
    n = np.arange(0, 1024)
    frequencies = n * 48000 / 2048
    for i in np.arange(start, end):
        axs[0].plot(frequencies[1:lim], measurements[i][0].real[1:lim], label=label[i])
        axs[1].plot(frequencies[1:lim], measurements[i][0].imag[1:lim], label=label[i])
        index = np.argmin(measurements[i][0].imag[0:lim])
        print(frequencies[index])
    axs[0].set_title("real part")
    axs[1].set_title("imaginary part")
    for ax in axs.flat:
        ax.grid(True)
        ax.set_xlabel("Frequency [Hz]")
        ax.set_ylabel("Amplitude [arbitrary unit]")
        ax.legend()
    plt.show()

compareSizes(measurements, 0, 3) # compare spectra for stainless steel
compareSizes(measurements, 3, 7) # compare spectra for copper rings

def getMinima(measurement, number):
    lim = 45
    frequency_minima = np.full(10, 0)
    value_at_freq_minima = np.full(10, 0, dtype=np.float32)
    for i in range(number):
        imag_array = measurement[i].imag[0:lim]
        index = np.argmin(imag_array)
        frequency_minima[i] = index * 48000 / 2048
        value_at_freq_minima[i] = imag_array[index]
    return frequency_minima, value_at_freq_minima

# Function to scale all the 10 measurements of one specific metal on to the same amplitude
def scaleToSameAmplitude(measurement, label, number):
    plt.figure()
    lim = 38
    n = np.arange(0, 1024)
    frequencies = n * 48000 / 2048
    index = np.argmin(measurement[0].imag[0:lim])
    max_0 = measurement[0].imag[index]
    for i in range(number):
        imag_array = measurement[i].imag[0:lim]
        index = np.argmin(imag_array)
        scale = max_0 / imag_array[index]
        plt.plot(frequencies[1:lim], scale * imag_array[1:lim], label=distance[i] + " cm")
    
    plt.legend(loc='lower left', bbox_to_anchor=(0., 0.), borderaxespad=0.)
    plt.grid(True)
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Amplitude [arbitrary unit]")
    plt.title(label)

for i in np.arange(0, 7):
    scaleToSameAmplitude(measurements[i], label=label[i], number=10)
plt.show()

def getAmplitudeAverage(measurement):
    lim = 45
    n = np.arange(0, 1024)
    frequencies = n * 48000 / 2048
    length = len(measurement[0].imag[1:lim])
    # print(length)
    value_imag = np.full(10, 0, dtype=np.float32)
    value_real = np.full(10, 0, dtype=np.float32)
    # print(measurement[0].imag[1:lim])
    for k in range(10):
        value_real_t = 0
        value_imag_t = 0
        for i in range(length):
            value_imag_t += measurement[k].imag[i]
            value_real_t += measurement[k].real[i]
        value_real[k] = value_imag_t / length
        value_imag[k] = value_real_t / length
    
    return np.abs(value_real), np.abs(value_imag)

value_real = np.full(10, 0, dtype=np.float32)
value_imag = np.full(10, 0, dtype=np.float32)
distance = [12.5, 17.5, 21.5, 25.5, 29.5, 33.5, 37.5, 41.5, 45.5, 49.5]

# Plot value at minimum for different distances
fig, axs = plt.subplots(1, 1, figsize=(15, 9))
for i in range(len(measurements)):
    frequency_minima, value_at_freq_minima = getMinima(measurements[i], 10)
    plt.plot([math.log10(x) for x in distance], [math.log10(x) for x in np.abs(value_at_freq_minima)], label=label[i])
    fit = stats.linregress([math.log10(x) for x in distance], [math.log10(x) for x in np.abs(value_at_freq_minima)])
    print(label[i])
    print(fit.slope)
    print(fit.stderr)
    
plt.grid(True)
plt.legend()
plt.ylabel("log(A) [arbitrary unit]")
plt.xlabel("log(d) [cm]")
plt.title("value at minimum for different distances")

# Plot average amplitude for different distances
fig, axs = plt.subplots(1, 2, figsize=(15, 9))
for i in range(len(measurements)):
    value_real, value_imag = getAmplitudeAverage(measurements[i])
    fit_real = stats.linregress([math.log10(x) for x in distance], [math.log10(x) for x in value_real])
    fit_imag = stats.linregress([math.log10(x) for x in distance], [math.log10(x) for x in value_imag])
    print("real " + label[i])
    print(fit_real.slope)
    print(fit_real.stderr)
    print("imag " + label[i])
    print(fit_imag.slope)
    print(fit_imag.stderr)
    axs[0].plot([math.log10(x) for x in distance], [math.log10(x) for x in value_real], label=label[i])
    axs[1].plot([math.log10(x) for x in distance], [math.log10(x) for x in value_imag], label=label[i])
    axs[0].set_title("real part")
    axs[1].set_title("imaginary part")

fig.subplots_adjust(wspace=0.5, hspace=0.5)  # Adjust these values as needed

for ax in axs.flat: 
    ax.grid(True)
    ax.set_xlabel("log(d) [cm]")
    ax.set_ylabel("log(A) [arbitrariy unit]")

plt.show()